Fill in the following variables to start

In [ ]:
%env SERPER_API_KEY=''
%env OPENAI_API_KEY=''
HF_USR_NAME = ''
TOOL_QA_ROOT = ''

### Upload to Huggingface

In [2]:
import pandas as pd
from datasets import Dataset
from datasets import DatasetDict

level = 'easy'
dataset = 'scirex'

dataset_dir = f'{dataset}-{level}.jsonl'
hf_dataset_name = f'toolqa_{dataset}_{level}'

df = pd.read_json(dataset_dir, lines=True)
df.head()

df['answer'] = df['answer'].apply(lambda x: str(x))
dataset = Dataset.from_pandas(df)

In [3]:
dataset_dict = DatasetDict({'train': dataset})
# push to hf for the ease for using dspy
dataset_dict.push_to_hub(repo_id=hf_dataset_name, private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/shirwu/toolqa_scirex_easy/commit/6856d5fcde7f21e44ca9e366cb0e172680a435f4', commit_message='Upload dataset', commit_description='', oid='6856d5fcde7f21e44ca9e366cb0e172680a435f4', pr_url=None, pr_revision=None, pr_num=None)

## Setting Up

* ToolQA

Before loading our datasets and going to the execution part, we'll need to configure the `lm` in `dspy.settings`. For the purpose of this notebook we'll be using `gpt-4o`.

In [4]:
import os
import dspy
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)


dspy.settings.configure(
    lm=dspy.OpenAI(
        model="gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY"),
        max_tokens=4000,
        temperature=0
    )
)

## Defining Signature

In [5]:
class ToolQASignature(dspy.Signature):
    """You will be given a question. Your task is to answer the question with a short response. 
    """
    
    question: str = dspy.InputField(
        prefix="Question:",
        desc="question to ask",
        format=lambda x: x.strip(),
    )
    answer: str = dspy.OutputField(
        prefix="Answer:",
        desc="answer to the question",
    )


## Loading Datasets

In [6]:
from random import sample
from dspy.datasets import DataLoader

dl = DataLoader()

In [7]:
tool_qa = dl.from_huggingface(
    f'{HF_USR_NAME}/' + hf_dataset_name,
    split="train",
    input_keys=("question", "answer"),
)

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [8]:
len(tool_qa)

100

In [9]:
import random
# set seed
random.seed(42)

train_idx = random.sample(range(len(tool_qa)), 40)
remaining_idx = list(set(range(len(tool_qa))) - set(train_idx))
test_idx = random.sample(remaining_idx, 60)

toolqa_train = [
    dspy.Example(question=example.question, answer=example.answer).with_inputs("question", "paper_id")
    for example in [tool_qa[i] for i in train_idx]
]
toolqa_test = [
    dspy.Example(question=example.question, answer=example.answer).with_inputs("question", "paper_id")
    for example in [tool_qa[i] for i in test_idx]
]

## Setting Up Tools

We'll setup `Avatar` modules for both signatures and all the `tools` can be used by each of the dataset. `Tool` is a pydantic model that Avatar expects the `tools` to be composed as more specifically it have 4 fields:

* `name` : Name of the tool
* `input_type` : Type of input the tool accepts
* `output_type` : Type of output the tool returns
* `tool` : The actual tool object

In [10]:
import os
import time
import uuid
import numpy as np
import jsonlines
from concurrent.futures import ProcessPoolExecutor
import sentence_transformers
import chromadb
from os import path as osp
from chromadb.config import Settings

EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
CHROMA_PERSIST_DIRECTORY = osp.join(TOOL_QA_ROOT, "data/chroma_db/scirex-v2")
CHROMA_COLLECTION_NAME = "all"
CHROMA_SERVER_HOST = "localhost"
CHROMA_SERVER_HTTP_PORT = "8000"
FILE_PATH = osp.join(TOOL_QA_ROOT, "data/external_corpus/scirex/Preprocessed_Scirex.jsonl")

def sentence_embedding(model, texts):
    embeddings = model.encode(texts)
    return embeddings

def create_chroma_db(chroma_server_host, chroma_server_http_port, collection_name):
    chroma_client = chromadb.Client(Settings(
        chroma_api_impl="rest",
        chroma_server_host=chroma_server_host,
        chroma_server_http_port=chroma_server_http_port,
    ))
    collection = chroma_client.get_or_create_collection(name=collection_name)
    return collection

def create_chroma_db_local(persist_directory, collection_name):
    chroma_client = chromadb.PersistentClient(path=persist_directory)
    collection = chroma_client.get_or_create_collection(name=collection_name)
    return collection

def insert_to_db(texts, model_name, cuda_idx, db):
    model = sentence_transformers.SentenceTransformer(model_name, device=f"cuda:{cuda_idx}")

    batch_embeddings = []
    batch_texts = []
    start_time = time.time()
    print(f"Total Articles to process: {len(texts)}, Current Thread: {cuda_idx}.")
    for i, text in enumerate(texts):
        # 2. generate embedding
        embeddings = sentence_embedding(model, text).tolist()

        batch_embeddings.append(embeddings)
        batch_texts.append(text)
        # 3. add to vectorstore per 500 articles or last article
        if i % 100 == 0 or i == len(texts)-1:
            batch_ids = [str(uuid.uuid1()) for _ in batch_texts]
            db.add(
                embeddings=batch_embeddings,
                documents=batch_texts,
                ids = batch_ids
            )
            batch_embeddings = []
            batch_texts = []
            print(f"Completed Processing article count: {i}, Current Thread: {cuda_idx}, Time took: {time.time() - start_time}.")
    print(f"Thread {cuda_idx} Completed. Total time took for thread: {time.time() - start_time}.")


# Multi-processing
def query_llm(query, is_local=True, start=None, end=None):
    cuda_idxes = [0]
    number_of_processes = len(cuda_idxes)
    input_texts = []
    db = create_chroma_db_local(CHROMA_PERSIST_DIRECTORY, CHROMA_COLLECTION_NAME)
    with open(FILE_PATH, 'r') as f:
        for item in jsonlines.Reader(f):
            input_texts.append(item["content"])
    # input_texts = np.array_split(input_texts, number_of_processes)

    args = ((input_texts[i], EMBED_MODEL_NAME, cuda_idxes[i], is_local) for i in range(number_of_processes))

    # if there is no file under the directory "/localscratch/yzhuang43/ra-llm/retrieval_benchmark/data/chroma_db/agenda", insert the data into the db
    # You should run insert_to_db the first time!
    if len(os.listdir(CHROMA_PERSIST_DIRECTORY)) == 0:
        insert_to_db(input_texts, model_name=EMBED_MODEL_NAME, cuda_idx=0, db=db)

    input_paths = np.array_split(input_texts, number_of_processes)
    with ProcessPoolExecutor(number_of_processes) as executor:
        executor.map(insert_to_db, args)
    model = sentence_transformers.SentenceTransformer(EMBED_MODEL_NAME, device=f"cuda:0")
    query_embedding = sentence_embedding(model, query).tolist()
    results = db.query(query_embeddings=query_embedding, n_results=3)
    retrieval_content = [result for result in results['documents'][0]]
    # print(retrieval_content)
    retrieval_content = '\n'.join(retrieval_content)
    return retrieval_content

query = "What is an atom"
print(query_llm(query))

paragraph : Sentence Level For representing a document , one can split it up into sentences , with each memory slot encoding one sentence . Both the key and the value encode the entire sentence as a bag - of - words . As the key and value are the same in this case , this is identical to a standard MemNN and this approach has been used in several papers .
paragraph : Window Level Documents are split up into windows of words ; in our tasks we only include windows where the center word is an entity . Windows are represented using bag - of - words . Window representations for MemNNs have been shown to work well previously . However , in Key - Value MemNNs we encode the key as the entire window , and the value as only the center word , which is not possible in the MemNN architecture . This makes sense because the entire window is more likely to be pertinent as a match for the question ( as the key ) , whereas the entity at the center is more pertinent as a match for the answer ( as the valu

In [11]:
from dspy.predict.avatar import Tool, Avatar
from langchain_community.utilities import GoogleSerperAPIWrapper, ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.tools import BaseTool, StructuredTool, tool

def RETRIEVE(query: str) -> str:
    """If you want to search for some paper information, you can use this tool and input a natural language query. For example, RETRIEVE(\'Which method achieves the highest PCK score?\') returns relevant paper paragraph and meta data."""
    return query_llm(query)

tools = [
    Tool(
        tool=StructuredTool.from_function(RETRIEVE),
        name="RETRIEVE",
        desc="If you want to search for some paper information, you can use this tool and input a natural language query. For example, RETRIEVE('Which method achieves the highest PCK score?') returns relevant paper paragraph and meta data."
    ),
    Tool(
        tool=GoogleSerperAPIWrapper(),
        name="WEB_SEARCH",
        desc="If you have a question, you can use this tool to search the web for the answer."
    ),
    Tool(
        tool=ArxivAPIWrapper(),
        name="ARXIV_SEARCH",
        desc="Pass the arxiv paper id to get the paper information.",
        input_type="Arxiv Paper ID",
    )
]

Once we have defined our `tools`, we can now create an `Avatar` object by passing the `tools` and `signature`. It takes 2 more optional parameters `verbose` and `max_iters`. `verbose` is used to display the logs and `max_iters` is used to control the number of iterations in multi step execution. 

An avatar agent stops the tool usage iteration once it reaches `max_iters` or when it prompts `Finish`. You can also create custom tools too, all you need to make sure is:

* You pass is a class object.
* Implements `__init__` and `run` method.
* Must take 1 string a input and returns 1 string as output.

If your tool doesn't return or takes input a string then you can make a custom wrapper to take care of that for now. In future we'll try to enable a diverse tool usage.

In [12]:
actor_agent = Avatar(
    tools=tools,
    signature=ToolQASignature,
    verbose=False,
)

Please use standard predictors, e.g. dspy.Predict and dspy.ChainOfThought.
They now support type annotations and other features of TypedPredictors and tend to work much better out of the box.
Please let us know if you face any issues: https://github.com/stanfordnlp/dspy/issues


## Evaluation

Open enden QA tasks are hard to evaluate on rigid metrics like exact match. So, we'll be using an improvised LLM as Judge for the evaluation of our model on test set.

In [13]:
class Evaluator(dspy.Signature):
    """Please act as an impartial judge to evaluate whether the answer is correct based on the ground truth answer"""
    
    question: str = dspy.InputField(
        prefix="Question:",
        desc="question to ask",
    )
    reference_answer: str = dspy.InputField(
        prefix="Ground Truth Answer:",
        desc="Ground truth answer to the question.",
    )
    answer: str = dspy.InputField(
        prefix="Answer:",
        desc="Answer to the question given by the model.",
    )
    rationale: str = dspy.OutputField(
        prefix="Rationale:",
        desc="Explanation of why the answer is correct or incorrect.",
    )
    is_correct: float = dspy.OutputField(
        prefix="Correct:",
        desc="Whether the answer is correct. Give 0 if incorrect, 1 if correct, (0, 1) if partially correct.",
    )


evaluator = dspy.TypedPredictor(Evaluator)


def metric(example, prediction, trace=None):  
    # We found sometimes the ground truth answers are incomplete or the answer
    # is part of the ground truth answer. Therefore, for better comparison, 
    # we use a continuous value for the correct score   
    acc = float(
        evaluator(
            question=example.question,
            answer=prediction.answer,
            reference_answer=example.answer
        ).is_correct
    ) 
    print(prediction.answer, '|', example.answer, '=>', acc)
    return acc

print(toolqa_train[0])
metric(toolqa_train[0], prediction=dspy.Example(answer='physics'))

 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


Example({'question': 'What is the corresponding Medium_Human-Normalized_Score score of the Ape-X method on Atari-57 dataset for Atari_Games task?', 'answer': '434.1%'}) (input_keys={'paper_id', 'question'})
physics | 434.1% => 0.0


0.0

For evaluation we can't use `dspy.Evaluate`, reason being that `Avatar` changes it's signature per iteration by adding the actions and it's results to it as fields. So we can create our own hacky thread safe evaluator for it.

In [14]:
import tqdm


os.environ["TOKENIZERS_PARALLELISM"] = "False"

from concurrent.futures import ThreadPoolExecutor

def process_example(example, signature):
    try:
        avatar = Avatar(
            signature,
            tools=tools,
            verbose=False,
        )
        prediction = avatar(**example.inputs().toDict())

        return metric(example, prediction)
    except Exception as e:
        print(e)
        return 0

# process_example(tool_qa[0], ToolQASignature)
def multi_thread_executor(test_set, signature, num_threads=60):
    total_score = 0
    total_examples = len(test_set)

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(process_example, example, signature) for example in test_set]

        for future in tqdm.tqdm(futures, total=total_examples, desc="Processing examples"):
            total_score += future.result()

    avg_metric = total_score / total_examples
    return avg_metric

def single_thread_executor(test_set, signature):
    total_score = 0
    total_examples = len(test_set)

    for example in tqdm.tqdm(test_set, desc="Processing examples"):
        total_score += process_example(example, signature)

    avg_metric = total_score / total_examples
    return avg_metric

In [17]:
teleprompter.thread_safe_evaluator(toolqa_test, actor_agent)

Processing examples:   0%|          | 0/60 [00:00<?, ?it/s]

The corresponding accuracy score of the DeepId2 method on the Labeled Faces in the Wild dataset for the Face Verification task is 99.15%. | 99.15% => 1.0


Processing examples:   2%|▏         | 1/60 [00:47<47:00, 47.81s/it]

The corresponding Accuracy score of the BB8 method on the LineMOD dataset for the 6D Pose Estimation task is 89.3%. | 83.9% => 0.0
The Percentage_correct score of the ResNet_ELU method on the CIFAR-100 dataset for the Image_Classification task is not readily available in the searched resources. | 73.5 => 0.0
The BLEU score of the ConvS2S method on the IWSLT2015 English-German dataset for the Machine Translation task is 26.73. | 26.73 => 1.0
The corresponding Accuracy score of the Planetoid method on the Cora dataset for the Document Classification task is 75.7%. | 75.7% => 1.0
The Inception score of the BigGAN-deep method on the ImageNet_128x128 dataset for the Conditional Image Generation task is 124.5. | 166.5 => 0.0
The F1 score of the Massively Multilingual Sentence Embeddings method on the BUCC French-to-English dataset for the Cross-Lingual Bitext Mining task is 93.91. | 93.91 => 1.0
The corresponding F1 score of the Neural-CRF_AE method on the CoNLL_2003 English dataset for the 

Processing examples:   3%|▎         | 2/60 [01:54<57:11, 59.17s/it]

...

Processing examples:  10%|█         | 6/60 [02:12<13:11, 14.67s/it]

The Frame__fps_ score for the CRF-RNN method on the Cityscapes dataset for the Real-Time Semantic Segmentation task is not directly available from the retrieved sources. Further specific searches or access to the original CRF-RNN paper or related publications might be necessary to find this specific metric. | 1.4 => 0.0


Processing examples: 100%|██████████| 60/60 [02:12<00:00,  2.21s/it]

The corresponding Accuracy score of the RetinaNet_Augmented_Autoencoders_ICP method on the T-LESS dataset for the 6D Pose Estimation task was not found in the available resources. It might be necessary to consult specific research papers or datasets that directly address this method and dataset combination. | 57.14 => 0.0


0.31666666666666665

In [ ]:
@

## Optimization

For the optimization of the `Actor` we'll be using `AvatarOptimizer`. It's a DSPy implementation of the [Avatar](https://github.com/zou-group/avatar/) method that optimizes the `Actor` for the given `tools` using a comparator module that optimizes Actor instruction. Note, that Actor is the Module that directs tool execution and flow, it's not the signature that we are passing. It doesn't optimize the instruction of the signature we pass. It takes the following parameters:

* `metric`: Metric that we'll be optimizing for
* `max_iters`: Maximum number of iterations for the optimizer
* `lower_bound`: Lower bound for the metric to classify example as negative
* `upper_bound`: Upper bound for the metric to classify example as positive
* `max_positive_inputs`: Maximum number of positive inputs sampled for comparator
* `max_negative_inputs`: Maximum number of negative inputs sampled for comparator
* `optimize_for`: Whether we want to maximize the metric or minimize it during optimization

Once the optimizer is done we can get the optimized actor and use it for the evaluation.

In [16]:
from dspy.teleprompt import AvatarOptimizer

teleprompter = AvatarOptimizer(
    metric=metric,
    max_iters=3,
    max_negative_inputs=10,
    max_positive_inputs=10,
    lower_bound=0.5,
    upper_bound=0.5
)

In [ ]:
optimized_actor_agent = teleprompter.compile(
    student=actor_agent,
    trainset=toolqa_train
)

Now we can evaluate our actor module, for this we've provided an implementation of thread safe evaluator that we above as part of class method of `AvatarOptimizer`.

In [ ]:
teleprompter.thread_safe_evaluator(toolqa_test, optimized_actor_agent)

Processing examples:   2%|▏         | 1/60 [00:11<11:11, 11.39s/it]

The corresponding Accuracy score of the BB8 method on the LineMOD dataset for the 6D Pose Estimation task is 89.3%. | 83.9% => 0.0
The corresponding AP score of the ACF-WIDER method on the WIDER Face Hard dataset for the Face Detection task is 89.7. | 0.290 => 0.0
The Inception score for the CEGAN-Ent-VI method on the CIFAR-10 dataset for the Image Generation task is 7.07. | 7.07 => 1.0
The Reasonable Miss Rate score of the Checkerboards method on the Caltech dataset for the Pedestrian Detection task is 18.5%. | 17.1 => 0.0
The corresponding Rank-1 score of the PDF method on the Market-1501 dataset for the Person Re-Identification task is 83.58%. | 84.14 => 0.0
The corresponding Validation_perplexity score of the AWD-LSTM-MoS___Partial_Shuffle method on the Penn_Treebank__Word_Level_ dataset for the Language_Modelling task is 53.92. | 55.89 => 0.0
The corresponding MAP score of the subCNN method on the PASCAL_VOC_2007 dataset for the Object Detection task is 68.5%. | 68.5% => 1.0
The B

Processing examples:   3%|▎         | 2/60 [00:27<13:47, 14.27s/it]

The Train Accuracy score of the 450D_DR-BiLSTM_Ensemble method on the SNLI dataset for the Natural Language Inference task is 94.8%. | 94.8 => 1.0
The UAS score for the Arc-hybrid method on the Penn Treebank dataset for the Dependency Parsing task was not found in the search results. | 93.56 => 0.0
The Percentage_correct score of the ResNet_ELU method on the CIFAR-100 dataset for the Image_Classification task could not be found using the available tools. | 73.5 => 0.0
The TAR at FAR=0.01 score for the Triplet probabilistic embedding method on the IJB-A dataset for the Face Verification task is not explicitly available in the retrieved documents. Further detailed search or access to specific datasets or publications may be required to obtain this information. | 90% => 0.0
The corresponding Mean IoU score of the ResNet-38_MS_COCO method on the PASCAL_VOC_2012 dataset for the Semantic Segmentation task is 84.9%. | 84.9% => 1.0
I was unable to find the Percentage_correct score for the ACN 

Processing examples:   7%|▋         | 4/60 [00:42<09:25, 10.10s/it]

The AUC score for the R2U-Net method on the LUNA dataset for the Lung Nodule Segmentation task could not be found in the available resources. | 0.9889 => 0.0
The MAP score for the aNMM method on the TrecQA dataset for the Question Answering task is not explicitly found in the search results. Further detailed search or access to specific academic papers or datasets might be required to obtain this information. | 0.750 => 0.0
The corresponding Aspect score of the LSTM-LOC method on the Sentihood dataset for the Aspect-Based Sentiment Analysis task is not explicitly found in the search results. However, the search results frequently mention LSTM-LOC in the context of the Sentihood dataset, suggesting it is a known method used in this area. Further detailed exploration of specific academic papers or datasets might be required to find the exact score. | 69.3 => 0.0


Processing examples: 100%|██████████| 60/60 [00:45<00:00,  1.33it/s]

The F-Measure score for the DeepFlux method on the SK-LARGE dataset for Object Skeleton Detection is not explicitly found in the search results. The search results consistently mention that DeepFlux outperforms other methods using the VGG16 backbone, but the exact F-Measure score is not provided in the available data. | 0.732 => 0.0


0.3333333333333333